In [1]:
### Define our Connector

import requests,os,time
def ratelimit(dt):
    "A function that handles the rate of your calls."
    time.sleep(dt) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30,waiting_time=0.5):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    self.waiting_time = waiting_time # define simple rate_limit parameter.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit(self.waiting_time)
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit(self.waiting_time)
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [2]:
connector = Connector('log_sds_Final_Project.csv') 


In [3]:
import requests 
from bs4 import BeautifulSoup


genre_list = ["Rock","Hip Hop"]  #Select a genre from the website @https://www.lyrics.com --> Pop, Hip Hop etc.  
num_pages = 2   # select manually the number of pages to be parsed

    
page_list = []   # a list containing the urls of 1 to N webpages based on the GENRE and Num_pages.
list_text = [] # initialize a list to store all the lyrics
list_title_songs = [] #initialise a list to store all the titles of the songs
list_year_songs = []  # a list to store the year or the song
list_genre = [] # static value based on Genre selected


for genre in genre_list:

    for i in range(1,num_pages+1):
        #loop and create a list of web-pages     
        page_list.append("https://www.lyrics.com/genres.php?genre=%s&p=%s" % (genre, str(i)))  #define genre manually in variable "genre".



    url_text = [] # a list containing URLs to the lyrics of different songs 
    base = 'https://www.lyrics.com'
    #lyrics_database= []


    for page in page_list:
        # for a single page from the page list iterate.

        url = page
        response,call_id = connector.get(url,'test_call') # using the connector object
        #r = requests.get( url )
        html = response.text    
        soup = BeautifulSoup(html, 'html') # parse the raw html using BeautifoulSoup
        lyrics_meta_title = soup.find_all("p", {'class': 'lyric-meta-title'}) # in the single page find all p tags with class "lyric-meta-title"

        for i in range(len(lyrics_meta_title)):

            lyric_url =  lyrics_meta_title[i].a.attrs['href'] # unpack the hyperlink from the p nodes
            url_text.append(base + lyric_url) # store the full URL for a single song   

            list_genre.append(genre)  # static value based on Genre selected
            
    #print(url_text)    

    
    
for url in url_text:
        #loop through all the songs' URLs.

    response, call_id = connector.get(url, 'project_call')
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    try:
        list_text.append(soup.find('pre',{"id" : 'lyric-body-text'}).text.replace("\r\n"," ").replace(",", " ")) #add a lyrics text to the collection

    except:
        list_text.append("NONE") #if input missing, add "NONE"
        pass 

    try:
        list_year_songs.append(soup.find('dd',{'class' : 'dd-margin'}).text)

    except:
        list_year_songs.append("NONE")
        pass
    try:
        list_title_songs.append(soup.find("h1",{'id' : 'lyric-title-text'}).text)

    except:
        list_title_songs.append("NONE")
        pass

        #list_genre.append(genre)  # static value based on Genre selected

   
    
    
    
    
    
    

In [4]:
#Checking whether the lists have equal number of elements.

print(len(list_text))
print(len(list_title_songs))
print(len(list_year_songs))
print(len(list_genre))

96
96
96
144


In [5]:
my_zip = list(zip(list_text,list_title_songs,list_year_songs, list_genre))

import pandas as pd

df = pd.DataFrame(my_zip, columns = ["Lyrics", "Title", "Year", "Genre"])


In [6]:
export_csv = df.to_csv("List_Songs_Lyrics.csv")